In [30]:
import json

In [23]:
with open('datasets/uniprot_do.json') as json_file:
    starting_do = json.load(json_file)

In [24]:
# Create dictionary do_id:[ancestor] ->do,
#   and do_id:depth -> depth,

do = {}  # { do_id : do_object }
obj = {}  # { id: do_id, name: definition, xref: list_of_omim_ids, is_a: list_of_parents, is_obsolete: True }
with open("res/doid.obo") as f:
    for line in f:
        line = line.strip().split(": ")
        if line and len(line) == 2 or line[0] == 'def':
            if len(line) > 2:
                k = line[0]
                v = line[1]
            else:
                k, v = line
            if k == "id" and v.startswith("DOID:"):
                obj["id"] = v.split(":")[1]
                if obj["id"] == '3488':
                    pass
            elif k == "xref" and "OMIM" in v:
                obj["omim"] = v.split(":")[1]
            elif k == "name":
                obj["name"] = v
            elif k == "is_a":
                obj.setdefault("is_a", []).append(v.split()[0].split(":")[1])
            elif k == "is_obsolete":
                obj["is_obsolete"] = True
        else:
            if obj.get("id") and not obj.get("is_obsolete"):
                do[obj["id"]] = obj
            obj = {}

In [3]:
doid_name = {}

for k, v in do.items():
    doid_name.setdefault(k, v['name'])

json = json.dumps(doid_name)
f = open("datasets/do_files/do_name.json", "w")
f.write(json)
f.close()

In [25]:
# Build an ancestors dictionary
ancestors = {}  # { term : list_of_ancestor_terms }
depth = {}
for node in do:
    c = 0
    node_ancestors = []
    node_parents = do[node].get("is_a")
    # Loop parent levels until no more parents
    while node_parents:
        c += 1
        node_ancestors.extend(node_parents)
        if "4" in node_parents:  # "4" is the root ID
            depth[node] = c
        # Get the parents of current parents (1 level up)
        node_parents = [term for parent in node_parents for term in do[parent].get("is_a", [])]
        # print(node_parents)
    ancestors[node] = set(node_ancestors)

In [26]:
for k,v in starting_do.items():
    for do_term in set(v):
        for ancestor in ancestors[do_term]:
            starting_do[k].append(ancestor)

In [27]:
for k,v in starting_do.items():
    starting_do[k]=list(set(v))

In [28]:
json = json.dumps(starting_do)
f = open("datasets/do_files/enriched_do.json", "w")
f.write(json)
f.close()

In [31]:
json = json.dumps(depth)
f = open("datasets/do_files/do_depth.json", "w")
f.write(json)
f.close()